<a href="https://colab.research.google.com/github/vineetsarpal/vs-gpt/blob/main/vs_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VSGPT: AI-Powered Guitar Pro Tab Generation

## Project Summary

This project demonstrates the creation of a Guitar Pro tab generation model using PyTorch and the DadaGP library. The core idea is to train a sequence-to-sequence model on existing Guitar Pro files, allowing it to learn musical patterns and generate new, stylistically similar tabs.

**Key Steps Involved:**

1.  **Data Preparation:** Guitar Pro (.gp) files are processed using the `dadagp.py` encoder to convert musical notation into a sequence of tokens (e.g., `tempo:120`, `note:s6:f-2`, `nfx:hammer`). These tokens represent various musical events, including notes, rests, techniques, and structural elements.
2.  **Vocabulary Building:** A unique vocabulary of all encountered tokens is built, and each token is mapped to a numerical ID. Special tokens like `<pad>` and `<unk>` are included for padding sequences and handling unknown tokens, respectively.
3.  **Dataset Creation:** Token sequences are prepared into a format suitable for sequence modeling, where the model learns to predict the next token based on previous tokens.
4.  **Model Definition (`VSGPTModel`):** An LSTM-based neural network is defined. It includes an embedding layer to convert token IDs into dense vectors, LSTM layers to capture sequential dependencies, and a linear output layer to predict the probability distribution over the next token in the vocabulary.
5.  **Model Training:** The model is trained using a standard language modeling objective (CrossEntropyLoss), optimizing its parameters to minimize the difference between predicted and actual next tokens. Training progress is monitored, and the best-performing model is saved.
6.  **Tab Generation:** Once trained, the model can generate new musical sequences by taking a prompt (e.g., `artist:vineet sarpal`, `tempo:120`, `start`) and iteratively predicting subsequent tokens. Techniques like temperature and top-k sampling are used to add creativity and coherence to the generation.
7.  **Decoding to Guitar Pro:** The generated token sequences are then passed to the `dadagp.py` decoder, which converts them back into playable Guitar Pro (.gp5) files, thus realizing the AI-generated music.
8.  **Output Management:** All generated tabs and saved model checkpoints are stored in a designated `outputs` and `models` directory on Google Drive for persistence.

This project provides a complete pipeline from raw Guitar Pro files to a trained generative model, capable of composing new musical pieces in a learned style.

## 1. Mount Google Drive and Setup Directories

In [ ]:
# ==== MOUNT GOOGLE DRIVE & PROJECT FOLDERS ====
from google.colab import drive
from google.colab import userdata
import os

drive.mount('/content/drive')

BASE_DIR = userdata.get('BASE_DIR') # set in secrets
TABS_DIR = os.path.join(BASE_DIR, 'tabs')
TOKENS_DIR = os.path.join(BASE_DIR, 'tokens')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
OUT_DIR = os.path.join(BASE_DIR, 'outputs')

os.makedirs(TABS_DIR, exist_ok=True)
os.makedirs(TOKENS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print("Base dir:", BASE_DIR)
print("Put your .gp files in:", TABS_DIR)

## 2. Data Preparation

In [ ]:
# ==== DISCOVER YOUR GUITAR PRO FILES ====
import glob

gp_files = sorted(glob.glob(TABS_DIR + '/*.gp*'))
print("Found", len(gp_files), "tabs:")
for f in gp_files:
    print(" -", os.path.basename(f))

In [ ]:
# ==== CUSTOM TOKENIZER FOR GUITAR PRO ====
#import guitarpro


# song = guitarpro.parse(gp_files[0])
# print(f"Looking at file: ", gp_files[0])
# tokens = []

# tokens.append('artist:vineet sarpal')
# tokens.append(f"tempo:{song.tempo}")
# tokens.append('start')

# # use first track (or tweak to select solo/lead)
# print(f"Song: ", song.artist)
# track = song.tracks[0]

# for track in song.tracks:
#   print(f"Track: ",track.strings)
#   for measure in track.measures:
#     for voice in measure.voices:
#       for beat in voice.beats:
#         for note in beat.notes:
#           tokens.append(f"note:{note}")
#           print(f"note:{note.value}, string:{note.string}")


In [ ]:
# ==== SETUP DADAGP ENCODER ====
!wget https://raw.githubusercontent.com/dada-bots/dadaGP/main/dadagp.py -O /content/dadagp.py
!wget https://raw.githubusercontent.com/dada-bots/dadaGP/main/token_splitter.py -O /content/token_splitter.py
!wget https://raw.githubusercontent.com/dada-bots/dadaGP/main/blank.gp5 -O /content/blank.gp5
!pip install "PyGuitarPro==0.6" --quiet
import subprocess, os

def encode_tab(gp_file):
    """Use DadaGP encoder to encode Guitar Pro tabs"""
    base_name = os.path.splitext(os.path.basename(gp_file))[0]
    tokens_file = os.path.join(TOKENS_DIR, f"{base_name}.tokens.txt")
    cmd = ['python', '/content/dadagp.py', 'encode', gp_file, tokens_file, 'vineet sarpal']

    result = subprocess.run(cmd, capture_output=True)
    if result.returncode == 0:
        with open(tokens_file, 'r') as f:
            tokens = f.read().strip().split()
        print(f"✅ Encoded {os.path.basename(gp_file)} → {tokens_file}")
        print(f"   {len(tokens)} tokens")
        return tokens
    else:
        print(f"❌ Failed to encode {os.path.basename(gp_file)}:")
        print(f"   {result.stderr}")
        return None


In [ ]:
# ==== ENCODE AND TOKENIZE YOUR GUITAR PRO FILES ====#
all_sequences = []
for gp_file in gp_files:
    tokens = encode_tab(gp_file)
    all_sequences.extend([tokens[i:i+512] for i in range(0, len(tokens)-512, 256)])

print(f"✅ Loaded {len(all_sequences)} sequences!")

## 3. Build Vocabulary

In [ ]:
# ==== BUILD VOCABULARY ====
from collections import Counter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

# Flatten all sequences to count tokens
all_tokens_flat = []
for seq in all_sequences:
    all_tokens_flat.extend(seq)

token_counts = Counter(all_tokens_flat)
vocab = ['<pad>', '<unk>'] + sorted(token_counts.keys())  # Add special tokens
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for token, idx in token_to_id.items()}
vocab_size = len(vocab)

print(f"✅ Vocab built: {vocab_size} tokens")
print("Most common:", token_counts.most_common(10))


## 4. Create Dataset

In [ ]:
# ==== CREATE DATASET ====
MAX_SEQ_LEN = 512

class TabDataset(Dataset):
    def __init__(self, sequences, token_to_id, max_len=MAX_SEQ_LEN):
        self.sequences = []
        for seq in sequences:
            # Convert to IDs
            ids = [token_to_id.get(t, token_to_id['<unk>']) for t in seq]
            if len(ids) >= 2:  # Need at least 2 tokens
                self.sequences.append(torch.tensor(ids, dtype=torch.long))

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        x = seq[:-1]  # input: all but last token
        y = seq[1:]   # target: all but first token (shifted)
        return x, y

dataset = TabDataset(all_sequences, token_to_id)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)  # Small batch for Colab

print(f"✅ Dataset ready: {len(dataset)} sequences")


## 5. Define Model Architecture

In [ ]:
# ==== DEFINE MODEL ====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class VSGPTModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2,
                           batch_first=True, dropout=0.3)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        logits = self.fc(lstm_out)
        return logits


In [ ]:
model = VSGPTModel(vocab_size).to(device)
print(f"✅ Model created: {sum(p.numel() for p in model.parameters())} parameters")

## 6.   Train Model

In [ ]:
# ==== TRAIN MODEL ====
# Training setup
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=token_to_id['<pad>'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

EPOCHS = 50
best_loss = float('inf')

print("🚀 Training model...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits.reshape(-1, vocab_size), y.reshape(-1))

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    scheduler.step(avg_loss)

    if avg_loss < best_loss:
        best_loss = avg_loss
        # Save best model to Drive
        torch.save({
            'model_state_dict': model.state_dict(),
            'token_to_id': token_to_id,
            'id_to_token': id_to_token,
            'vocab_size': vocab_size
        }, os.path.join(MODELS_DIR, 'vs_gpt_best.pt'))

    print(f"Epoch {epoch+1:02d}/{EPOCHS} | Loss: {avg_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.2e}")

print("✅ Training complete! Best model saved.")


## 7. Generate and Decode Tabs

In [ ]:
# ==== GENERATE NEW TABS ====
def generate_tabs(model, prompt_tokens, max_new=256, temperature=0.8, top_k=50):
    """Generate new guitar tabs from your style"""
    model.eval()
    ids = [token_to_id.get(t, token_to_id['<unk>']) for t in prompt_tokens]

    with torch.no_grad():
        for _ in range(max_new):
            # Use last 256 tokens as context
            context = torch.tensor([ids[-256:]], device=device)
            logits = model(context)[:, -1, :] / temperature

            # Top-K sampling
            top_k_logits, top_k_ids = torch.topk(logits, top_k)
            probs = torch.softmax(top_k_logits, dim=-1)
            next_id = top_k_ids[0][torch.multinomial(probs[0], 1)].item()

            ids.append(next_id)

            # Stop conditions
            if id_to_token[next_id] in ['end', 'end_of_song']:
                break

    return [id_to_token[i] for i in ids]

In [ ]:
# Test generation!
prompt = ['vineet', 'sarpal', 'downtune:-2', 'tempo:120', 'start']
generated_tokens = generate_tabs(model, prompt, max_new=1024, temperature=0.85)

print("\n🎸 GENERATED TAB (first 50 tokens):")
print(" ".join(generated_tokens[:50]))
print(f"\nTotal length: {len(generated_tokens)} tokens")

In [ ]:
# ==== TEMP SAVE GENERATED TOKENS ====
# Save generated tokens to a temp file
generated_tokens_file = '/content/generated_song.tokens.txt'
with open(generated_tokens_file, 'w') as f:
    f.write(" ".join(generated_tokens))

print(f"✅ Saved tokens to {generated_tokens_file}")

In [ ]:
# ==== DECODE TOKENS TO GP FILES ====
def decode_to_guitarpro(tokens_file, output_gp_file):
    """Convert tokens back to Guitar Pro using DadaGP decoder"""
    import subprocess

    cmd = ['python', '/content/dadagp.py', 'decode', tokens_file, output_gp_file]
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✅ Decoded {tokens_file} → {output_gp_file}")
        return output_gp_file
    else:
        print(f"❌ Decode failed:")
        print(result.stderr)
        return None


In [ ]:
# Generate and decode
output_gp = '/content/generated_tab.gp5'
decoded_file = decode_to_guitarpro(generated_tokens_file, output_gp)

if decoded_file:
    print(f"🎸 PLAYABLE TAB CREATED: {decoded_file}")

In [ ]:
# ==== SAVE GENERATED TABS ====
# Copy to your project folder
drive_output = os.path.join(OUT_DIR, 'song_1.gp5')
!cp {output_gp} {drive_output}
print(f"✅ Saved to Drive: {drive_output}")

In [ ]:
# COMPLETE: Generate → Decode → Save
def generate_and_decode(model, prompt, output_name="generated_tab", max_new=1024):
    """Full pipeline: tokens → Guitar Pro → Drive"""

    # Generate
    tokens = generate_tabs(model, prompt, max_new=max_new, temperature=0.85)
    temp_tokens = '/content/temp.tokens.txt'

    with open(temp_tokens, 'w') as f:
        f.write("\n".join(tokens)) # Changed to join with newlines

    # Decode
    temp_gp = '/content/temp.gp5'
    decoded = decode_to_guitarpro(temp_tokens, temp_gp)

    if decoded:
        # Save to Drive
        final_gp = os.path.join(OUT_DIR, f"{output_name}.gp5")
        import shutil
        shutil.copy(temp_gp, final_gp)
        print(f"🎵 CREATED: {final_gp}")
        print(f"Tokens: {len(tokens)} | Techniques: {sum(1 for t in tokens if 'nfx' in t)}")
        return final_gp

    return None

In [ ]:
# Create 3 variations
prompts = [
    ['artist:vineet sarpal', 'tempo:120', 'start'],
    ['artist:vineet sarpal', 'tempo:140', 'downtune:0', 'start'],
    ['artist:vineet sarpal', 'tempo:100', 'start']
]

for i, prompt in enumerate(prompts):
    generate_and_decode(model, prompt, f"vs_gpt_variation_{i+1}")

## 8. Save Model

In [ ]:
# ==== SAVE MODEL ====
# Save final model + full workspace
model_path = os.path.join(MODELS_DIR, 'vs_gpt_complete.pt')
torch.save({
    'model_state_dict': model.state_dict(),
    'token_to_id': token_to_id,
    'id_to_token': id_to_token,
    'all_sequences': all_sequences  # Your original data
}, model_path)

print("✅ Everything saved to Drive!")
print("\nNext time, load with:")
print(f"checkpoint = torch.load('{model_path}')")


In [ ]:
# ======================================================================================= #

 ### NOTE: If you have already trained your model and saved the checkpoint, you do not need to re-run the entire training pipeline.

 Follow these steps to set up your environment, load the model, and generate new tabs:

* Mount Google Drive and Setup Directories
* Setup DadaGP Encoder
* Define Model Architecture
* Define Generation Function
* Define Decoding Function
* Define Complete Generate & Decode Pipeline
* Load the Trained Model and Vocabulary
* Generate New Tabs

## 9. Load Model

In [ ]:
# Load the checkpoint
print(f"Loading model from: {MODELS_DIR}")
model_path = os.path.join(MODELS_DIR, 'vs_gpt_complete.pt')
checkpoint = torch.load(model_path, map_location=device)

# Reconstruct necessary components from the checkpoint
loaded_token_to_id = checkpoint['token_to_id']
loaded_id_to_token = checkpoint['id_to_token']
loaded_vocab_size = len(loaded_token_to_id)

# Instantiate the model architecture
loaded_model = VSGPTModel(loaded_vocab_size).to(device)

# Load the model state dictionary
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval() # Set the model to evaluation mode


print("✅ Model loaded successfully!")
print(f"Loaded model has {sum(p.numel() for p in loaded_model.parameters())} parameters")

# Set the global variables
token_to_id = loaded_token_to_id
id_to_token = loaded_id_to_token
vocab_size = loaded_vocab_size

print("✅ Global variables updated successfully!")

## 10. Generate New Tabs

In [ ]:
print("\nGenerating a new tab using the loaded model...")
new_prompt = ['artist:vineet sarpal', 'downtune:-2', 'tempo:120', 'start']
generate_and_decode(loaded_model, new_prompt, max_new=1024)